In [26]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from math import sqrt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
%matplotlib inline

In [5]:
original_df = pd.read_excel('/home/quark/Documents/projects/TFM/Databases/data.xlsx')
original_df.sort_values(['Timestamp'], ascending=True, inplace=True)
original_df['Timestamp'] = pd.to_datetime(original_df['Timestamp'], unit='s')

marag_df = original_df[original_df['SiteId'] == 2][['Timestamp', 'TotalEntries']]
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)
marag_df = marag_df['2017-01-09 00:00:00':]
#Since from this date the time series is constinuous, we slice our dataframe consequently

In [6]:
def create_lags(dataframe, N):
    temp_df = dataframe.copy()
    for i in range(1,N+1):
        temp_df['t-' + str(i)] = temp_df['TotalEntries'].shift(i)
    
    return temp_df.iloc[N:]
df_with_lags = create_lags(marag_df, 48)

In [7]:
#Initial data preparation:
def train_dev_test_split(X_data, y_data):
    
##CODE IMPROVEMENT: raise error if X and y of different size!    
    cut1 = int(0.6*len(X_data))
    cut2 = ((len(X_data)-cut1)//2) + cut1
    X_train = X_data[:cut1]
    X_dev = X_data[cut1:cut2]
    X_test = X_data[cut2:]
    y_train = y_data[:cut1]
    y_dev = y_data[cut1:cut2]
    y_test = y_data[cut2:]
    
    return X_train, X_dev, X_test, y_train, y_dev, y_test
    
#X_train, X_temp, y_train, y_temp = train_test_split(men_df.drop('TotalEntriesMen', axis = 1), men_df['TotalEntriesMen'], test_size = 0.4, random_state = 42, shuffle=False)
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(df_with_lags.drop(['TotalEntries'], axis=1), df_with_lags['TotalEntries'])

In [12]:
model = LinearRegression().fit(X_train, y_train)

X_train_raw = marag_df[:X_train[-1:].index.astype(str)[0]]

In [27]:

#   !!!   DOCUMENT THE FUNCTION PROPERLY

def walk_forward_validation(X_train, y_train, X_test, L, model):

    '''This function integrates a walk-forward validation method for multiple models.
    Parameters: model, #lags, train data, test data, length of test_data'''

    X_grow = X_train_raw.astype('float64').sort_index()
    y_grow_hat = []
    #RMSE_record = []
    for i in tqdm(range(len(X_test))):
        y_hat = model.predict(X_test[i:i+1])
        #RMSE_record.append(y_hat)
        y_grow_hat.append(y_hat)
        y_hat = pd.DataFrame(y_hat, index=X_test[i:(i+1)].index, columns=['TotalEntries'])
        #y_hat = pd.DataFrame(y_hat, columns=['TotalEntries'])
        #y_hat.set_index(X_dev[i:(i+1)].index, inplace=True)
        X_grow = pd.concat([X_grow, y_hat])
        X_data = create_lags(X_grow, L)
        model.fit(X_data.drop(['TotalEntries'], axis=1), X_data['TotalEntries'])
        #y_hat_trans = create_lags(pd.DataFrame(y_hat), 49)
        #X_grow = pd.concat(X_grow, y_hat_trans)
        #model.fit(X_grow)
    
    
    return y_grow_hat


In [ ]:
total_predictions = walk_forward_validation(X_train, y_train, X_dev, 48, model)

 17%|█▋        | 1297/7450 [01:02<04:56, 20.75it/s]

In [18]:
np.sqrt(mean_squared_error(y_dev, total_predictions))

43.416341408547105

In [25]:
r2_score(y_dev, total_predictions)

0.9025640812584325

In [15]:
len(total_predictions)

7450

In [16]:
type(total_predictions)

list

In [17]:
total_predictions[:10]

[array([365.18346681]),
 array([425.87873676]),
 array([422.87122209]),
 array([422.23035053]),
 array([425.30593206]),
 array([278.44811364]),
 array([333.69039303]),
 array([394.73703152]),
 array([492.8074945]),
 array([374.84400718])]